In [2]:
pip install transformers torch torchvision opencv-python fer librosa soundfile numpy pandas


  Using cached fer-22.5.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux

In [8]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00


In [22]:
!pip install resampy



In [14]:
pip install gradio transformers deepface opencv-python librosa scikit-learn pandas matplotlib


In [12]:
pip install deepface


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=98832e6c3e60d23aa9f1d14675bc02555fca391dd2ec788f99162ad5241ced15
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [24]:

import os
import zipfile
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report
import joblib
import requests
from tqdm import tqdm

# ----------------------------------------
# 1. Download RAVDESS Dataset
# ----------------------------------------
def download_ravdess():
    url = "https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip"
    filename = "ravdess.zip"

    if not os.path.exists("ravdess"):
        print("Downloading RAVDESS...")
        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(filename, 'wb') as f:
                for chunk in tqdm(r.iter_content(chunk_size=8192)):
                    f.write(chunk)

        print("Unzipping...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall("ravdess")
        os.remove(filename)

# ----------------------------------------
# 2. Feature Extraction (no resampy)
# ----------------------------------------
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    if len(y) < sr * 3:
        y = np.pad(y, (0, sr * 3 - len(y)))
    y = y[:sr * 3]
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

# ----------------------------------------
# 3. Load Data
# ----------------------------------------
def load_data():
    emotions = {
        '01': 'neutral',
        '02': 'calm',
        '03': 'happy',
        '04': 'sad',
        '05': 'angry',
        '06': 'fearful',
        '07': 'disgust',
        '08': 'surprised'
    }

    X, y = [], []
    for root, _, files in os.walk("ravdess"):
        for file in files:
            if file.endswith(".wav"):
                emotion_code = file.split("-")[2]
                emotion = emotions.get(emotion_code)
                if emotion:
                    path = os.path.join(root, file)
                    features = extract_features(path)
                    X.append(features)
                    y.append(emotion)
    return np.array(X), np.array(y)

# ----------------------------------------
# 4. Train Model and Save
# ----------------------------------------
def train_model():
    download_ravdess()
    X, y = load_data()

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("\n🎯 Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))

    joblib.dump(model, "speech_emotion_model.pkl")
    joblib.dump(scaler, "scaler.pkl")
    print("✅ Model and scaler saved!")

# ----------------------------------------
# Run
# ----------------------------------------
if __name__ == "__main__":
    train_model()



🎯 Classification Report:
              precision    recall  f1-score   support

       angry       0.71      0.61      0.66        33
        calm       0.63      0.93      0.75        41
     disgust       0.63      0.56      0.59        43
     fearful       0.56      0.54      0.55        35
       happy       0.49      0.50      0.49        34
     neutral       0.82      0.50      0.62        18
         sad       0.56      0.55      0.55        44
   surprised       0.62      0.60      0.61        40

    accuracy                           0.61       288
   macro avg       0.63      0.60      0.60       288
weighted avg       0.61      0.61      0.60       288

✅ Model and scaler saved!


In [26]:
import gradio as gr
import librosa
import numpy as np
import pandas as pd
import os
from datetime import datetime
from transformers import pipeline
from deepface import DeepFace


text_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")


def detect_text_emotion(text):
    result = text_classifier(text)
    return result[0]['label']

def extract_audio_features(file_path):
    y, sr = librosa.load(file_path)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    return np.mean(mfcc.T, axis=0)

def detect_audio_emotion(audio_path):
    features = extract_audio_features(audio_path)
    return "happy" if np.mean(features) > 0 else "sad"


def detect_image_emotion(image_path):
    try:
        analysis = DeepFace.analyze(img_path=image_path, actions=['emotion'], enforce_detection=False)
        return analysis[0]['dominant_emotion']
    except Exception as e:
        return "Error analyzing image"


def recommend_task(emotion):
    mapping = {
        "happy": "Work on creative projects",
        "sad": "Take a short break or do light tasks",
        "angry": "Avoid team meetings, focus on solo tasks",
        "neutral": "Proceed with daily assigned tasks",
        "fear": "Engage in supportive team discussions",
        "disgust": "Review and restructure task workload",
        "surprise": "Participate in team idea sharing"
    }
    return mapping.get(emotion.lower(), "Do general tasks")


def log_and_alert(user, emotion):
    df = pd.DataFrame([[user, emotion, datetime.now()]], columns=["User", "Emotion", "Timestamp"])
    df.to_csv("mood_logs.csv", mode='a', header=not os.path.exists("mood_logs.csv"), index=False)

    if os.path.exists("mood_logs.csv"):
        logs = pd.read_csv("mood_logs.csv")
        recent = logs.tail(3)
        if len(recent) >= 3 and all(recent["Emotion"].str.lower() == emotion.lower()):
            return f"⚠️ HR ALERT: Prolonged '{emotion}' detected!"
    return ""


def handle_text_input(text):
    emotion = detect_text_emotion(text)
    task = recommend_task(emotion)
    alert = log_and_alert("Employee1", emotion)
    return emotion, task, alert

def handle_audio_input(audio_path):
    emotion = detect_audio_emotion(audio_path)
    task = recommend_task(emotion)
    alert = log_and_alert("Employee1", emotion)
    return emotion, task, alert

def handle_image_input(image_path):
    emotion = detect_image_emotion(image_path)
    task = recommend_task(emotion)
    alert = log_and_alert("Employee1", emotion)
    return emotion, task, alert


with gr.Blocks() as app:
    gr.Markdown("# 🤖 Zidio AI-Powered Task Optimizer")
    gr.Markdown("Analyze emotions from text, audio, and images. Get smart task suggestions and HR alerts.")

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(label="Enter Employee's Text Input")
            text_btn = gr.Button("Analyze Text")
        with gr.Column():
            audio_input = gr.Audio(type="filepath", label="Upload Audio (.wav)")
            audio_btn = gr.Button("Analyze Audio")
        with gr.Column():
            image_input = gr.Image(type="filepath", label="Upload Face Image")
            image_btn = gr.Button("Analyze Image")

    output_emotion = gr.Text(label="Detected Emotion")
    output_task = gr.Text(label="Recommended Task")
    output_alert = gr.Text(label="HR Alert (if any)")

    text_btn.click(fn=handle_text_input, inputs=text_input, outputs=[output_emotion, output_task, output_alert])
    audio_btn.click(fn=handle_audio_input, inputs=audio_input, outputs=[output_emotion, output_task, output_alert])
    image_btn.click(fn=handle_image_input, inputs=image_input, outputs=[output_emotion, output_task, output_alert])

app.launch()


Device set to use cpu


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cbceaf5b7b735f592a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
